# HTTPS & TLS
von Jens Gräfe, Matthias Rinker & Jan Kühnemund

# Inhaltsverzeichnis

# HTTPS

# HTTPS ist HTTP über TLS

HTTP nutzt als Transport Protokoll TCP. HTTPS setzt stattdessen auf TLS auf, welches auch TCP nutzt. Dadurch ist das komplette HTTP Paket verschlüsselt. (rfc2818[2][2.2])

# HTTP Paketaufbau

In [4]:
from diagrams import Diagram
from diagrams.onprem.client import Client
from diagrams.onprem.network import Tyk

with Diagram("HTTP Header Format"):
    client = Client("Client")
    http = Tyk("HTTP")
    client >> http

# HTTP Paketaufbau

# HTTP Paketaufbau

# TLS Frame

# Geschichte von TLS

# TLS 1.1

# TLS 1.2

# TLS 1.3

# HTTPS Demo

In [ ]:
import ssl
import socket
import OpenSSL

# Set the target website
website = "www.google.com"

# Check the certificate of the website
context = ssl.create_default_context()
s = socket.create_connection((website, 443))
ssl_sock = context.wrap_socket(s, server_hostname=website)
cert = ssl_sock.getpeercert()

# Print the certificate
print(cert)